# Tidy Data

In [255]:
import pandas as pd
from pydataset import data
import seaborn as sns
import numpy as np

### Exercise 1: Attendance Data

- Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

- You should end up with something like this:


| name | |
| --- | --- |
| Billy | 0.5250 |
| Jane | 0.6875 |
| John | 0.9125 |
| Sally | 0.7625 |
Name: grade, dtype: float64

In [351]:
#read csv and save as a dataframe
attendance = pd.read_csv('untidy-data/attendance.csv')
#rename column 'Unnamed: 0' as name
attendance = attendance.rename(columns = {'Unnamed: 0': 'name'})
#replace alpha characters as their respective numeric values 
attendance = attendance.replace({'P': 1, 'A': 0, 'H': .5, 'T': .9})
#create a new column called 'attend_percent' and save the percentage of the students attendance
attendance['attend_percent'] = attendance.sum(axis=1)/(len(attendance.columns)-1)
#group by the name to show the students in alpabetical order
attendance = attendance.groupby('name').sum()
#dropping columns 0-8 to only show attend_percent
attendance.drop(attendance.iloc[:, 0:8], inplace = True, axis = 1)
#displaying dataframe
attendance

,attend_percent
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


In [352]:
#alt sol, creating fucntion to replace letter grades
def get_attendance_grade(letter):
    if letter == 'P':
        return 1
    elif letter == 'A':
        return 0
    elif letter == 'T':
        return .9
    elif letter == 'H':
        return .5

In [354]:
#alt sol
df = pd.read_csv('untidy-data/attendance.csv')
df = df.rename(columns = {'Unnamed: 0': 'student'})
df = df.melt(id_vars = 'student', var_name = 'date')
df['grade'] = df.value.apply(get_attendance_grade)
df.groupby('student').grade.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

### Exercise 2: Coffee Levels

- A) Read the coffee_levels.csv file.

In [124]:
#read csv and save as a dataframe
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
#dispay dataframe
coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


- B) Transform the data so that each carafe is in it's own column.

In [123]:
#creating a pivot table with the values as 'coffee_amount'
#the rows will be listed as 'hour'
#the columns will be listed as 'coffee_carafe'
coffee.pivot_table('coffee_amount', 'hour', 'coffee_carafe')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


- C) Is this the best shape for the data?

If we were going by the definition then it is not the best shape. Although this is easier to read by the eye, the other graph allows us to use our functions and compare quickly using groupby.

### Exercise 3: Cake Recipes

- A) Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [165]:
#read csv and save it as a dataframe
cakes = pd.read_csv('untidy-data/cake_recipes.csv')
cakes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


- B) Tidy the data as necessary.

In [215]:
# change all the columns into rows while keeping the index variable as 'recipe:position'
cake = cakes.melt(id_vars = 'recipe:position')
#create a new dataframe that holds the column 'recipe:position' into 2 different columns
recipe_and_position_df = cakes['recipe:position'].str.split(':', expand = True)
#name those columns 'recipe' and 'position' respectively
recipe_and_position_df.columns = ['recipe', 'position']
#multiply the rows by 4 so there will be an even number or rows to combine in the next step
recipe_and_position_df = pd.concat([recipe_and_position_df] * 4, ignore_index = True)
#combine recipe_and_position_df and cake 
df = pd.concat([recipe_and_position_df, cake], axis = 1)
#drop the 'recipe:position' column
df = df.drop(columns = 'recipe:position')
#rename the columns variable and value as temp and rating respectively
df = df.rename(columns = {'variable': 'temp', 'value':'rating'})
df

,recipe,position,temp,rating
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


In [ ]:
#alt sol, the correct order to perform the task above
recipe_and_position_df = cakes['recipe:position'].str.split(':', expand = True)
recipe_and_position_df.columns = ['recipe', 'position']
df = pd.concat([recipe_and_position_df, cake], axis = 1)
df = df.drop(columns = 'recipe:position')
df.melt(id_vars['recipe','position'], var_name = 'temp', value_name = 'rating')

- C) Which recipe, on average, is the best?

In [217]:
#getting the mean of the of the individual recipes
df.groupby('recipe').mean()

#df.groupby('recipe').rating.mean()

,rating
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


Recipe b is the best on average

- D) Which oven temperature, on average, produces the best results?

In [218]:
#getting the mean of the temperatures
df.groupby('temp').mean()

#df.groupby('temp').rating.mean()

,rating
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


The temperature 275 produced the best results on average.

- E) Which combination of recipe, rack position, and temperature gives the best result?

In [262]:
#retrieving the row that identifies the max rating
df.loc[[df.rating.idxmax()]]

#df.sort_values(by = 'rating').tail()
#this is good practice to compare values, not just see the answer

,recipe,position,temp,rating
26,b,bottom,300,99.248541


other methods to reshape data
